## Task 1: Sentiment analysis of twitter and stockwits data

In [401]:
import json
from pprint import pprint
import matplotlib as mpl
import matplotlib.pyplot as pltfrom 

from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
import numpy as np
import pandas as pd

from keras.layers import Flatten
from keras import regularizers

### Loading data from JSON

In [402]:
with open('Microblog_Trainingdata.json') as data_file:    
    Microblog_Trainingdata = json.load(data_file)

for i in range(0,len(Microblog_Trainingdata)):
    Microblog_Trainingdata[i]['spans']=Microblog_Trainingdata[i]['spans'][0]
    Microblog_Trainingdata[i]['sentiment score']=float(Microblog_Trainingdata[i]['sentiment score'])
#pprint(Microblog_Trainingdata)
with open('Microblog_Trialdata.json') as data_file:   
    Microblog_Trialdata = json.load(data_file)
    

#pprint(data)
for i in range(0,len(Microblog_Trialdata)):
    Microblog_Trialdata[i]['spans']=Microblog_Trialdata[i]['spans'][0]
    Microblog_Trialdata[i]['sentiment score']=float(Microblog_Trialdata[i]['sentiment score'])

In [403]:
with open('Microblogs_Testdata.json') as data_file:   
    Microblogs_Testdata = json.load(data_file)
    
#pprint(Microblog_Testdata)

In [404]:
for i in Microblog_Trialdata:
    Microblog_Trainingdata.append(i)

In [405]:
def lower_keys(x):
    if isinstance(x, list):
        return [lower_keys(v) for v in x]
    elif isinstance(x, dict):
        return dict((k.lower(), v.lower() if isinstance(v, str) else v) for k, v in x.items())
    else:
        return x
Microblog_Trainingdata=lower_keys(Microblog_Trainingdata)

In [406]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [407]:
import string
def stringClean( inputString ) :
    return inputString.translate(str.maketrans('','',string.punctuation)).encode("ascii", errors="ignore").decode()

In [408]:
def removeStopWords(inputString):
    splitString = inputString.split()
    for word in stop:
        for s in splitString:
            if s == word:
                splitString.remove(s)
    return " ".join(splitString)

In [409]:
from sklearn.metrics.pairwise import cosine_similarity
def calculateCosineSimilarity( pVal,gVal ):
    similarity = cosine_similarity(pVal,gVal)
    pNorm = np.linalg.norm(pVal)
    gNorm = np.linalg.norm(gVal)
    wt = pNorm/gNorm
    return wt * similarity[0][0]

### Loading Testing and Testing data

In [410]:
maxlen = 0

X_train = []
X_trial=[]
Y_trial=[]
X_test = [Microblog['spans'] for Microblog in Microblogs_Testdata]
Y_train = []
Y_test = []
output=[]
for Microblog in Microblog_Trainingdata :
    X_train.append(removeStopWords(stringClean(Microblog['spans'])))
    Y_train.append(0 if Microblog['sentiment score']<=0 else 1)
    output.append(Microblog['sentiment score'])
    if maxlen < len(Microblog['spans'].split()) :
        maxlen = len(Microblog['spans'].split())

#for Microblog in Microblog_Trialdata :
 #   X_train.append(Microblog['spans'])
  #  Y_train.append(0 if Microblog['sentiment score']<=0 else 1)
   # if maxlen < len(Microblog['spans'].split()) :
    #    maxlen = len(Microblog['spans'].split())


print(len(X_train)+len(X_test), len(Y_train))


2510 1710


### Selecting top 15000 words using tokenizer library in keras

In [411]:
tokenizer = Tokenizer(num_words=maxlen*20)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
data = pad_sequences(sequences, maxlen)
data.shape

(1710, 25)

### LSTM Model

In [412]:
model = Sequential()
model.add(Embedding(maxlen*20, 32, input_length=maxlen))
#model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='leakyrelu'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())
model.add(LSTM(32, dropout=0.3, recurrent_dropout=0.15))
#, kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01)
model.add(Dense(4, activation='tanh'))
model.add(Dense(1, activation='tanh'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 25, 32)            16000     
_________________________________________________________________
lstm_15 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_39 (Dense)             (None, 4)                 132       
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 5         
Total params: 24,457
Trainable params: 24,457
Non-trainable params: 0
_________________________________________________________________


In [413]:
history = model.fit(data, np.array(Y_train), validation_split=0.2, epochs=15,verbose = 1)

Train on 1368 samples, validate on 342 samples
Epoch 1/15
1368/1368 [==============================] - 4s - loss: 1.2524 - acc: 0.4320 - val_loss: 0.6698 - val_acc: 0.6374
Epoch 2/15
1368/1368 [==============================] - 1s - loss: 0.6591 - acc: 0.6316 - val_loss: 0.6472 - val_acc: 0.6404
Epoch 3/15
1368/1368 [==============================] - 1s - loss: 0.6466 - acc: 0.6447 - val_loss: 0.6274 - val_acc: 0.6550
Epoch 4/15
1368/1368 [==============================] - 1s - loss: 0.6119 - acc: 0.6908 - val_loss: 0.6036 - val_acc: 0.6667
Epoch 5/15
1368/1368 [==============================] - 1s - loss: 0.5683 - acc: 0.7390 - val_loss: 0.5520 - val_acc: 0.7515
Epoch 6/15
1368/1368 [==============================] - 1s - loss: 0.4960 - acc: 0.8019 - val_loss: 0.4926 - val_acc: 0.8041
Epoch 7/15
1368/1368 [==============================] - 1s - loss: 0.4465 - acc: 0.8216 - val_loss: 0.4933 - val_acc: 0.7778
Epoch 8/15
1368/1368 [==============================] - 1s - loss: 0.3887 - ac

In [414]:
sequences = tokenizer.texts_to_sequences(np.array(X_train))
data = pad_sequences(sequences, maxlen)
predictions = model.predict(data)
for i in range(len(predictions)):
    print(Microblog_Trainingdata[i]['spans'],Microblog_Trainingdata[i]['sentiment score'],"%.3f" % predictions[i])

watching for bounce tomorrow 0.366 0.879
record number of passengers served in 2015 0.638 0.968
out $nflx -.35 -0.494 -0.160
looking for a strong bounce 0.46 0.964
very intrigued with the technology and growth potential 0.403 0.972
short worked 0.0 -0.309
overbought -0.296 -0.491
absolute garbage still up -0.546 0.567
biggest market losers -0.438 -0.528
love this company long time. 0.408 0.835
$goog $googl would suck -0.398 -0.049
who won't pay anymore -0.349 -0.601
no edge offered 0.025 0.958
runs into the 50sma on the acquisition news 0.486 0.634
t can't go down 0.308 0.815
now seems like its helping the downtrend -0.372 -0.449
mastered their supply chain 0.461 0.619
most bullish stocks on twitter during this dip 0.408 0.973
big dumping -0.699 0.662
trade continuing very nicely from yesterday 0.495 0.971
in from 3.61 0.306 0.619
stochastic overbought -0.385 -0.656
unusual call buying 0.336 0.971
reserves are in decline 0.279 0.619
all on a longer-term, technical swing long basis. 0.5

oversold 0.43 0.948
double top reject -0.341 0.890
added. 0.47 0.945
working on cuba deals 0.419 0.953
dividend hike green light 0.316 0.976
i am excited about all these plays 0.322 0.645
commodity weakness -0.376 0.563
$fb small flong 109.25 via calls 0.352 0.952
had a nice breakout of the recent consolidation 0.452 0.968
sells flat..i am out.. -0.541 0.619
airplane and hospitality industries set their sights on #cuba 0.405 0.967
reserves are in decline 0.279 0.619
this pull back of -0.03 0.453
pie in the sky 0.365 0.966
first quarter is amazing 0.322 0.976
$tfm will have a way to go price wise to compete with kroger. $kr 0.129 0.945
sector stocks leading today 0.454 0.966
biggest st50 gainers 0.452 0.931
bears take a clear near term advantage -0.501 -0.107
will a onetime zombie stock rise again? 0.241 0.914
today's bull flag 0.2 0.912
iphone se could be doing better than expected 0.395 0.882
now up to 200 stocks making new 52 week highs 0.458 0.978
would short $amzn on any rally -0.6

excellent / factual bear argument -0.507 -0.509
bears take a clear near term advantage -0.501 -0.107
relative weakness is pretty apparent -0.262 0.563
don't think buyout rumor strong enough too be driving this up just yet -0.02 0.725
$yhoo +2.61% 0.16 0.886
$tplm triangle #petroleum 9.80+3% 2m expanding caliber 0.376 0.912
downbeaten stocks may be ripe for buys 0.316 0.976
downgrade -0.463 -0.533
isrg short -0.678 -0.309
$pozn +30 0.146 0.046
-2.10% vs -0.90% est -0.489 0.619
daily mail enters competition to acquire yah 0.342 -0.031
stocks higher in afternoon trade 0.357 0.945
watchlist top stocks 0.342 0.961
wants to ta -0.23 0.512
$viab down 40% -0.516 0.619
sudden optimism about iphone sales 0.279 0.224
google, ibm, arm and amd begin the assault on intel's server franchise 0.368 0.704
monday sold -0.44 -0.457
earning play $fast 47 puts, $vrx 34 puts , $bidu 182.5 puts -0.387 -0.769
bullish ma crossovers 0.457 0.974
watchlist top stocks 0.342 0.961
looks constructive 0.28 0.793
$unt 

In [415]:
df=pd.DataFrame.from_dict(Microblog_Trainingdata)

In [416]:
df['Predicted Sentiment Score']=predictions
df.head()

,cashtag,id,sentiment score,source,spans,Predicted Sentiment Score
0,$fb,719659409228451840,0.366,twitter,watching for bounce tomorrow,0.878914
1,$luv,719904304207962112,0.638,twitter,record number of passengers served in 2015,0.968236
2,$nflx,5329774,-0.494,stocktwits,out $nflx -.35,-0.160244
3,$dia,719891468173844480,0.460,twitter,looking for a strong bounce,0.964293
4,$plug,20091246,0.403,stocktwits,very intrigued with the technology and growth ...,0.971810


In [417]:
task1Dataframe=pd.DataFrame({"ID":df['id'],"Spans":df['spans'],"Cashtag":df['cashtag'],"Sentiment Score":df['sentiment score'],"Predicted Sentiment Score":df['Predicted Sentiment Score']})
task1Dataframe=task1Dataframe[['ID','Spans','Cashtag','Sentiment Score','Predicted Sentiment Score']]
task1Dataframe

,ID,Spans,Cashtag,Sentiment Score,Predicted Sentiment Score
0,719659409228451840,watching for bounce tomorrow,$fb,0.366,0.878914
1,719904304207962112,record number of passengers served in 2015,$luv,0.638,0.968236
2,5329774,out $nflx -.35,$nflx,-0.494,-0.160244
3,719891468173844480,looking for a strong bounce,$dia,0.460,0.964293
4,20091246,very intrigued with the technology and growth ...,$plug,0.403,0.971810
5,5819749,short worked,$gmcr,0.000,-0.308639
6,709741154393133056,overbought,$ibm,-0.296,-0.490966
7,17892972,absolute garbage still up,$josb,-0.546,0.567474
8,709834259687710720,biggest market losers,$cstm,-0.438,-0.527917
9,708481442079068160,love this company long time.,$pypl,0.408,0.835396


In [418]:
#Convert dataframe to csv file
task1Dataframe.to_csv('Task1.csv',index=False)

### Computing Cosine score

In [419]:
from sklearn.metrics.pairwise import cosine_similarity

In [420]:
predictedVals = [pr[0] for pr in predictions ]

In [421]:
import numpy as np
output=np.array(output).reshape(1,-1)
predictedVals=np.array(predictedVals).reshape(1,-1)
similarity=cosine_similarity(predictedVals,output)
print(similarity)

[[ 0.69175178]]
